In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, r'../../invert')
import mne
import pickle as pkl

from time import time

from invert.forward import get_info
from invert.simulate import generator
from invert import Solver
from scipy.sparse import csr_matrix
import os
from config import forward_models
# os.makedirs('D:/data/flex_ssm/evaluation', exist_ok=True)
os.makedirs("predictions/", exist_ok=True)
n_jobs = 7
n_reg_params = 10
base_path = "D:/data/flex_ssm"

# Load Forward Models

In [2]:
import os
import numpy as np
fwd_model_path = os.path.join(base_path, "forward_models/")
models = {
    "Clean Coarse": fwd_model_path + "fsaverage_biosemi256_Clean_coarse-fwd.fif",
    "Clean Fine": fwd_model_path + "fsaverage_biosemi256_Clean_fine-fwd.fif",
    "Ratio-50 Fine": fwd_model_path + "fsaverage_biosemi256_Ratio-50_fine-fwd.fif",
    "Ratio-30 Fine": fwd_model_path + "fsaverage_biosemi256_Ratio-30_fine-fwd.fif",
    }

with open(os.path.join(fwd_model_path, "info_biosemi256.pkl"), 'rb') as f:
    info = pkl.load(f)

fwds = dict()
infos = dict()
for name, model in models.items():
    fwd = mne.read_forward_solution(model, verbose=0)
    fwd = mne.convert_forward_solution(fwd, force_fixed=True)
    fwds[name] = fwd
    
    
del fwd

    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]


# Define Solvers

In [3]:
diffusion_parameter = 0.

solver_dicts = [
    # {
    #     "name": "SSM",
    #     "display_name": "SSM_-3",
    #     "solver_args": {},
    #     "make_args": {
    #         "n_orders": 0,
    #         "refine_solution": True,
    #         "max_iter": 6,
    #         "lambda_reg1": 1e-3,
    #         "alpha": "auto", 
    #         "n": 2, 
    #         "k": 2, 
    #     },
    #     "apply_args": {
            
    #     },
    #     "recompute_make": True
    # },
    # {
    #     "name": "AP",
    #     "display_name": "AP",
    #     "solver_args": {},
    #     "make_args": {
    #         "n_orders": 0,
    #         "refine_solution": True,
    #         "max_iter": 6,
    #         "alpha": "auto", 
    #         "n": 2, 
    #         "k": 2, 
    #     },
    #     "apply_args": {
            
    #     },
    #     "recompute_make": True
    # },
    # {
    #     "name": "RAP-MUSIC",
    #     "display_name": "RAP-MUSIC",
    #     "solver_args": {},
    #     "make_args": {
    #         "n_orders": 0,
    #         "refine_solution": False,
    #         "stop_crit": 0.,
    #         "alpha": "auto", 
    #         "n": 2, 
    #         "k": 2, 
    #     },
    #     "apply_args": {},
    #     "recompute_make": True
    # },
    {
        "name": "MCMV",
        "display_name": "MCMV",
        "solver_args": {},
        "make_args": {
            "alpha": 1e-2
        },
        "apply_args": {},
        "recompute_make": True
    },
    {
        "name": "Champagne",
        "display_name": "Champagne",
        "solver_args": {
            "update_rule": "convexity"
        },
        "make_args": {
            "alpha": 5e-8,
            "convergence_criterion": 1e-9
        },
        "apply_args": {},
        "recompute_make": True
    }
]

solvers_names = [s["name"] for s in solver_dicts]

# Figure 1 - 3

In [14]:
from funs import predict_sources_parallel3

# sim_condition = "figure-1"
sim_conditions = [
    "figure-1",
    "figure-2",
    # "figure-3",
]

solver_dicts = solver_dicts

for sim_condition in sim_conditions:
    print("Sim Condition: ", sim_condition)
    # Load the data set
    fullpath = os.path.join(base_path, "simulations", f"sim_{sim_condition}.pkl")
    with open(fullpath, "rb") as f:
        x_test, y_test, sim_info, _ = pkl.load(f)
    if sim_condition == "figure-2":
        indices = np.where((sim_info['snr'] == 0) & (sim_info['inter_source_correlations'] == 0.9))[0]
        x_test = x_test[indices]
        y_test = y_test[indices]
        sim_info = sim_info.iloc[indices]
    
    print(f"number of samples now: {x_test.shape[0]}")


    # Predict sources with each forward model
    for i, (fwd_name, fwd) in enumerate(fwds.items()):
        print("Forward Model: ", fwd_name)
        
        # Check if predicted file exists already:
        filename = os.path.join(base_path, "predictions", f"""sim_and_preds_{sim_condition}_{fwd_name.replace(" ", "-")}_non-greedy.pkl""")
        
        if os.path.isfile(filename):
            print("\texists already, continuing")
            continue
        else:
            print(f"\tcomputing file {filename}")
        # print(f"\tcomputing file {filename}")

        # Prepare (or re-initialize) solvers
        for solver_dict in solver_dicts:
            solver_dict["solver"] = Solver(solver_dict["name"], n_reg_params=n_reg_params, **solver_dict["solver_args"])
        
        # Parallel compute inverse solutions
        res = predict_sources_parallel3(solver_dicts, fwd, info, x_test, sim_info, n_jobs=n_jobs)
        # Organize/ Store Inverse solutions and simulations
        solver_names = [sd["display_name"] for sd in solver_dicts]
        stc_dict = {sd["display_name"]: [] for sd in solver_dicts}
        proc_time_make = {sd["display_name"]: [] for sd in solver_dicts}
        proc_time_apply = {sd["display_name"]: [] for sd in solver_dicts}
        
        for sample in res:
            for solver, data in sample[0].items():
                stc_dict[solver].append(data)
            for solver, t in sample[1].items():
                proc_time_make[solver].append(t)
            for solver, t in sample[2].items():
                proc_time_apply[solver].append(t)
        
        with open(filename, 'wb') as f:
            pkl.dump([stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply], f)

del stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply

Sim Condition:  figure-1
number of samples now: 2800
Forward Model:  Clean Coarse
	exists already, continuing
Forward Model:  Clean Fine
	exists already, continuing
Forward Model:  Ratio-50 Fine
	exists already, continuing
Forward Model:  Ratio-30 Fine
	exists already, continuing
Sim Condition:  figure-2
number of samples now: 2600
Forward Model:  Clean Coarse
	computing file D:/data/flex_ssm\predictions\sim_and_preds_figure-2_Clean-Coarse_non-greedy.pkl
Forward Model:  Clean Fine
	computing file D:/data/flex_ssm\predictions\sim_and_preds_figure-2_Clean-Fine_non-greedy.pkl
Forward Model:  Ratio-50 Fine
	computing file D:/data/flex_ssm\predictions\sim_and_preds_figure-2_Ratio-50-Fine_non-greedy.pkl


c:\Users\lukas\virtualenvs\ssm\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Forward Model:  Ratio-30 Fine
	computing file D:/data/flex_ssm\predictions\sim_and_preds_figure-2_Ratio-30-Fine_non-greedy.pkl


# Figure 1 - Misestimation for 3 true sources

In [5]:
from funs import predict_sources_parallel3
import pandas as pd

# sim_condition = "figure-1"
sim_conditions = ["figure-1", "figure-12"]
n_sources_assumed = [1, 2, 3, 4, 5]
for sim_condition in sim_conditions:
    # Load the data set
    fullpath = f"simulations/sim_{sim_condition}.pkl"
    with open(fullpath, "rb") as f:
        x_test, y_test, sim_info, _ = pkl.load(f)

    x_test_new = []
    y_test_new = []
    sim_info_new = []
    n_sources = 3
    inter_source_correlation_list = (0.1, 0.5, 0.9, 0.95)
    for i in range(sim_info.shape[0]):
        if (sim_info.iloc[i, ]["n_sources"] == n_sources) and (sim_info.iloc[i, ]["inter_source_correlations"] in inter_source_correlation_list):
            
            x_test_new.append(x_test[i])
            y_test_new.append(y_test[i])
            sim_info_new.append(sim_info.iloc[i])
        # break
    x_test = np.stack(x_test_new, axis=0)
    y_test = np.stack(y_test_new, axis=0)
    sim_info = pd.DataFrame(sim_info_new)

    for n_sources in n_sources_assumed:
        # Predict sources with each forward model
        for i, (fwd_name, fwd) in enumerate(fwds.items()):
            print("Forward Model: ", fwd_name)
            
            # Check if predicted file exists already:
            filename = f"predictions/sim_and_preds_{sim_condition}_{fwd_name}-{int(n_sources)}-assumed.pkl"
            print(filename)
            if os.path.isfile(filename):
                print("\texists already, continuing")
                continue
            
            # Prepare (or re-initialize) solvers
            for solver_dict in solver_dicts:
                solver_dict["solver"] = Solver(solver_dict["name"], n_reg_params=n_reg_params)
            
            # Parallel compute inverse solutions
            res = predict_sources_parallel3(solver_dicts, fwd, info, x_test, sim_info, n_jobs=n_jobs, n_sources=n_sources)
            # Organize/ Store Inverse solutions and simulations
            solver_names = [sd["display_name"] for sd in solver_dicts]
            stc_dict = {sd["display_name"]: [] for sd in solver_dicts}
            proc_time_make = {sd["display_name"]: [] for sd in solver_dicts}
            proc_time_apply = {sd["display_name"]: [] for sd in solver_dicts}
            
            for sample in res:
                for solver, data in sample[0].items():
                    stc_dict[solver].append(data)
                for solver, t in sample[1].items():
                    proc_time_make[solver].append(t)
                for solver, t in sample[2].items():
                    proc_time_apply[solver].append(t)

            with open(filename, 'wb') as f:
                pkl.dump([stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply], f)

    del stc_dict, x_test, y_test, sim_info, proc_time_make, proc_time_apply

Forward Model:  Clean Coarse
predictions/sim_and_preds_figure-1_Clean Coarse-1-assumed.pkl
	exists already, continuing
Forward Model:  Clean Fine
predictions/sim_and_preds_figure-1_Clean Fine-1-assumed.pkl
	exists already, continuing
Forward Model:  Ratio-50 Fine
predictions/sim_and_preds_figure-1_Ratio-50 Fine-1-assumed.pkl
	exists already, continuing
Forward Model:  Ratio-30 Fine
predictions/sim_and_preds_figure-1_Ratio-30 Fine-1-assumed.pkl
	exists already, continuing
Forward Model:  Clean Coarse
predictions/sim_and_preds_figure-1_Clean Coarse-2-assumed.pkl
	exists already, continuing
Forward Model:  Clean Fine
predictions/sim_and_preds_figure-1_Clean Fine-2-assumed.pkl
	exists already, continuing
Forward Model:  Ratio-50 Fine
predictions/sim_and_preds_figure-1_Ratio-50 Fine-2-assumed.pkl
	exists already, continuing
Forward Model:  Ratio-30 Fine
predictions/sim_and_preds_figure-1_Ratio-30 Fine-2-assumed.pkl
	exists already, continuing
Forward Model:  Clean Coarse
predictions/sim_and

NameError: name 'stc_dict' is not defined